# NMDS Gradients

In class, we performed NMDS by initializing points for each object (i.e., animal) randomly and then iteratively moving points in a direction that will reduce stress. In this assignment, we will implement the calculation of these directions along with the rest of the NMDS algorithm.

You will need to refer to the lecture notebooks from class as well as what you learned in discussion section to complete this assignment.

In [113]:
!pip install -q otter-grader

import otter
grader = otter.Notebook("hw6.ipynb")

import numpy as np
import pandas as pd

The human similarity data is loaded below. We want to find a set of points that is related to these similarity ratings in the in the way that Shepard's Law predicts. That set of points is an inferred psychological representation.

In [114]:
sim_vals = [
 [1., 0.600554459474065, 0.7536383164437648, 0.5312856091329679],
 [0.600554459474065, 1., 0.49306869139523984, 0.7288934141100247],
 [0.7536383164437648, 0.49306869139523984, 1., 0.4088417197978041],
 [0.5312856091329679, 0.7288934141100247, 0.4088417197978041, 1.]
]
labels = ['dog', 'cat', 'wolf', 'rabbit']
df_sim = pd.DataFrame(sim_vals, columns=labels, index=labels)
df_sim

,dog,cat,wolf,rabbit
dog,1.000000,0.600554,0.753638,0.531286
cat,0.600554,1.000000,0.493069,0.728893
wolf,0.753638,0.493069,1.000000,0.408842
rabbit,0.531286,0.728893,0.408842,1.000000


First we will randomly initialize points for each animal:

In [115]:
# do not change
np.random.seed(10)

# do not change
def create_initial_random_points():
    points = np.random.rand(4, 2)
    points_df = pd.DataFrame(
        points, 
        columns=['dim1', 'dim2'], 
        index=['dog', 'cat', 'wolf', 'rabbit']
    )
    return points_df

# do not change
df_guesses = create_initial_random_points()
df_guesses

,dim1,dim2
dog,0.771321,0.020752
cat,0.633648,0.748804
wolf,0.498507,0.224797
rabbit,0.198063,0.760531


As discussed in section, NMDS iteratively adjusts each point $x_i$ with respect to each other point $x_j$ (where $i \neq j$) using the formula:

$x_i = x_i + \text{step\_size} * (d̂ᵢⱼ - dᵢⱼ) × (x_j - x_i)/d̂ᵢⱼ$,

where $dᵢⱼ$ is distance in psychological space, $d̂ᵢⱼ$ is the distance between points we are adjusting, and $(d̂ᵢⱼ - dᵢⱼ) × (x_j - x_i)/d̂ᵢⱼ$ is called the **gradient**.

To review only briefly, $(x_j - x_i)/d̂ᵢⱼ$ is a unit vector that points from $x_i$ to $x_j$, and $(d̂ᵢⱼ - dᵢⱼ)$ is the important signed term in the stress that determines (1) whether we step in the direction of $(x_j - x_i)/d̂ᵢⱼ$ or $-(x_j - x_i)/d̂ᵢⱼ$, and (2) the size of the step we take relative to other points.

**Exercise 1:**

Perform NMDS using the following criteria:

- Store gradients for all points on all iterations in a multidimensional numpy array called `directions` with shape `(n_iterations, n_animals, n_animals - 1, 2)`.
- Set $\text{step\_size}$ to $0.4$.
- Run for $100$ iterations.
- Store the stress for each iteration in an array called `stress_vals`.

In [116]:
# Your code goes here
step_size = 0.4
n_iterations = 100
n_animals = 4
stress_vals = []
directions = np.ndarray((n_iterations, n_animals, n_animals - 1, 2))

df_d = -np.log(df_sim)

for iter in range(n_iterations):
    gradients = np.zeros((n_animals, n_animals - 1, 2))
    points = df_guesses.values
    stress = 0
    
    for i in range(n_animals):
        grad_iter = 0
        
        for j in range(n_animals):
            if i == j:
                continue
            
            xi, xj = points[i], points[j]
            
            d_hat_ij = np.sqrt(np.pow(xi[0] - xj[0], 2)+ np.pow(xi[1] - xj[1],2))
            
            d_ij = df_d.iloc[i, j]
            
            unit_vector = (xj - xi) / d_hat_ij
            
            gradient = (d_hat_ij - d_ij) * unit_vector
            gradients[i, grad_iter, :] = gradient
            grad_iter += 1
            
            if i < j:
                stress += (d_ij - d_hat_ij) ** 2


    for i in range(n_animals):
        total_gradient = np.sum(gradients[i], axis=0)
        points[i] += step_size * total_gradient

    df_guesses.iloc[:, :] = points

    stress_vals.append(stress)
    directions[iter] = gradients

# DO NOT CHANGE
for stress_val in stress_vals:
    print(stress_val)

0.2691402532488104
0.02608335510530766
0.00863644611828238
0.005313252072251314
0.004266024167864888
0.0038443891641416006
0.0036368668607966426
0.00351072165417821
0.003421623163095012
0.0033513715154942034
0.003292516685924377
0.003241146740784444
0.0031951598872577394
0.0031531390577189036
0.0031141205620522894
0.0030773568209189186
0.003042267749488096
0.003008369888209929
0.0029752560061458763
0.002942566378830945
0.002909976374516819
0.002877181808190113
0.0028438902837475163
0.0028098121483923835
0.002774653834592824
0.002738111272348268
0.0026998643655338035
0.0026595717212045595
0.002616866114908463
0.002571350577663119
0.0025225955941837377
0.0024701378050366184
0.0024134810093358336
0.002352100416779865
0.0022854514622615983
0.0022129847354130728
0.002134168819280518
0.002048522839303872
0.0019556602096256963
0.0018553441578268944
0.0017475539099486334
0.001632557731854966
0.0015109854036131146
0.0013838885840282663
0.0012527739328650285
0.0011195924160101074
0.0009866708135

In [117]:
grader.check("q1")

q1 results: All test cases passed!